# --- Day 6: Guard Gallivant ---

The Historians use their fancy device again, this time to whisk you all away to the North Pole prototype suit manufacturing lab... in the year 1518! It turns out that having direct access to history is very convenient for a group of historians.

You still have to be careful of time paradoxes, and so it will be important to avoid anyone from 1518 while The Historians search for the Chief. Unfortunately, a single guard is patrolling this part of the lab.

Maybe you can work out where the guard will go ahead of time so that The Historians can search safely?

You start by making a map (your puzzle input) of the situation. For example:

```
....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#...
```

The map shows the current position of the guard with `^` (to indicate the guard is currently facing up from the perspective of the map). Any obstructions - crates, desks, alchemical reactors, etc. - are shown as `#`.

Lab guards in 1518 follow a very strict patrol protocol which involves repeatedly following these steps:

1. If there is something directly in front of you, turn right 90 degrees.
2. Otherwise, take a step forward.

Following the above protocol, the guard moves up several times until she reaches an obstacle (in this case, a pile of failed suit prototypes):

```
....#.....
....^....#
..........
..#.......
.......#..
..........
.#........
........#.
#.........
......#...
```

Because there is now an obstacle in front of the guard, she turns right before continuing straight in her new facing direction:

```
....#.....
........>#
..........
..#.......
.......#..
..........
.#........
........#.
#.........
......#...
```

Reaching another obstacle (a spool of several very long polymers), she turns right again and continues downward:

```
....#.....
.........#
..........
..#.......
.......#..
..........
.#......v.
........#.
#.........
......#...
```

This process continues for a while, but the guard eventually leaves the mapped area (after walking past a tank of universal solvent):

```
....#.....
.........#
..........
..#.......
.......#..
..........
.#........
........#.
#.........
......#v..
```

By predicting the guard's route, you can determine which specific positions in the lab will be in the patrol path. Including the guard's starting position, the positions visited by the guard before leaving the area are marked with an `X`:

```
....#.....
....XXXXX#
....X...X.
..#.X...X.
..XXXXX#X.
..X.X.X.X.
.#XXXXXXX.
.XXXXXXX#.
#XXXXXXX..
......#X..
```

In this example, the guard will visit **41 distinct positions** on your map.

---

### Task

Predict the path of the guard. How many distinct positions will the guard visit before leaving the mapped area?

In [54]:
from typing import Generator
from enum import Enum
from dataclasses import dataclass, field
import numpy as np


class Direction(str, Enum):
    UP = "^"
    DOWN = "v"
    LEFT = "<"
    RIGHT = ">"

    @property
    def offset(self):
        """Return the (dx, dy) offset for this direction."""
        return {
            Direction.UP: (0, -1),
            Direction.DOWN: (0, 1),
            Direction.LEFT: (-1, 0),
            Direction.RIGHT: (1, 0),
        }[self]

    @property
    def turn_right(self):
        """Return the direction obtained by turning 90 degrees to the right."""
        return {
            Direction.UP: Direction.RIGHT,
            Direction.RIGHT: Direction.DOWN,
            Direction.DOWN: Direction.LEFT,
            Direction.LEFT: Direction.UP,
        }[self]


@dataclass
class Position:
    x: int
    y: int

    def __hash__(self):
        return hash((self.x, self.y))

    def __eq__(self, other: "Position"):
        return self.x == other.x and self.y == other.y

    def __add__(self, other: "Position") -> "Position":
        return Position(self.x + other.x, self.y + other.y)

    def __sub__(self, other: "Position") -> "Position":
        return Position(self.x - other.x, self.y - other.y)

    def __repr__(self):
        return f"({self.x}, {self.y})"


@dataclass
class TurningPoint:
    position: Position
    direction: Direction

    def __hash__(self):
        return hash((self.position, self.direction))


@dataclass
class Map:
    grid: np.ndarray

    @classmethod
    def from_file(cls, filepath: str) -> "Map":
        with open(filepath, "r", encoding="utf-8") as f:
            grid = np.array([list(line.strip()) for line in f])
        return cls(grid=grid)

    def is_in_bounds(self, position: Position) -> bool:
        return (
            0 <= position.y < self.grid.shape[0]
            and 0 <= position.x < self.grid.shape[1]
        )

    def is_obstacle(self, position: Position) -> bool:
        return self.grid[position.y, position.x] == "#"

    def add_obstacle(self, position: Position) -> "Map":
        self.grid[position.y, position.x] = "#"
        return self


@dataclass
class Guard:
    map: Map
    history: list[Position] = field(default_factory=list)
    direction: Direction = Direction.UP
    turning_points: list[TurningPoint] = field(default_factory=list)
    look_for_loops: bool = False
    safe_obstacle_positions: list[Position] = field(default_factory=list)
    evil_obstacle_positions: list[Position] = field(default_factory=list)

    @property
    def position(self) -> Position:
        """Current position is always the last in history."""
        return self.history[-1]

    def move(self):
        """Move the guard or handle obstacles."""
        while True:
            new_position = self.next_position()

            # Check if the new position is valid
            if not self.map.is_in_bounds(new_position):
                return False

            if self.map.is_obstacle(new_position):
                self.turn_right_90deg()
                continue  # Retry with new direction
            else:
                # check if adding an obstacle creates a loop
                self.simulate_obstruction_at(new_position)

            # Add the new position to history
            self.history.append(new_position)
            return True

    def next_position(self) -> Position:
        """Next position based on the current position and direction."""
        offset = Position(*self.direction.offset)
        return self.position + offset

    def turn_right_90deg(self):
        """Turn 90 degrees to the right."""
        self.direction = self.direction.turn_right
        self.turning_points.append(
            TurningPoint(position=self.position, direction=self.direction)
        )

    def patrol(self) -> Generator["Guard", None, None]:
        """Continue moving until the guard leaves the map."""
        while self.move():
            yield self

    def simulate_obstruction_at(self, position: Position) -> None:
        if not self.look_for_loops:
            return
        if position == self.history[0]:
            # can't place obstacle at guard's starting point
            return
        if (
            position in self.safe_obstacle_positions
            or position in self.evil_obstacle_positions
        ):
            return

        fake_guard = Guard(
            map=Map(grid=self.map.grid.copy()).add_obstacle(position),
            history=self.history.copy(),
            direction=self.direction,
            turning_points=self.turning_points.copy(),
        )
        for state in fake_guard.patrol():
            if len(state.turning_points) != len(set(state.turning_points)):
                # loop detected since we have repeated turning points
                self.evil_obstacle_positions.append(position)
                return
        self.safe_obstacle_positions.append(position)

    @classmethod
    def from_map(cls, map: Map) -> "Guard":
        """Initialize the guard from the map."""
        for direction in Direction:
            locations = np.argwhere(map.grid == direction.value)
            if locations.size > 0:
                y, x = locations[0]  # First occurrence
                return cls(map=map, history=[Position(x, y)], direction=direction)
        raise ValueError(f"No guard found in the map: {map.grid}")


game_map = Map.from_file("./example.txt")
guard = Guard.from_map(game_map)

# Perform the patrol step-by-step
for state in guard.patrol():
    continue
print(f"{len(set(guard.history))=}")

len(set(guard.history))=41


In [55]:
game_map = Map.from_file("./input.txt")
guard = Guard.from_map(game_map)

# Perform the patrol step-by-step
for state in guard.patrol():
    continue
print(f"{len(set(guard.history))=}")

len(set(guard.history))=4559


### Part Two

While The Historians begin working around the guard's patrol route, you borrow their fancy device and step outside the lab. From the safety of a supply closet, you time travel through the last few months and record the nightly status of the lab's guard post on the walls of the closet.

Returning after what seems like only a few seconds to The Historians, they explain that the guard's patrol area is simply too large for them to safely search the lab without getting caught.

Fortunately, they are pretty sure that adding a single new obstruction won't cause a time paradox. They'd like to place the new obstruction in such a way that the guard will get stuck in a loop, making the rest of the lab safe to search.

To have the lowest chance of creating a time paradox, The Historians would like to know all of the possible positions for such an obstruction. The new obstruction can't be placed at the guard's starting position - the guard is there right now and would notice.

In the above example, there are only 6 different positions where a new obstruction would cause the guard to get stuck in a loop. The diagrams of these six situations use `O` to mark the new obstruction, `|` to show a position where the guard moves up/down, `-` to show a position where the guard moves left/right, and `+` to show a position where the guard moves both up/down and left/right.

#### Option 1: Put a printing press next to the guard's starting position
```
....#.....
....+---+#
....|...|.
..#.|...|.
....|..#|.
....|...|.
.#.O^---+.
........#.
#.........
......#...
```

#### Option 2: Put a stack of failed suit prototypes in the bottom right quadrant of the mapped area
```
....#.....
....+---+#
....|...|.
..#.|...|.
..+-+-+#|.
..|.|.|.|.
.#+-^-+-+.
......O.#.
#.........
......#...
```

#### Option 3: Put a crate of chimney-squeeze prototype fabric next to the standing desk in the bottom right quadrant
```
....#.....
....+---+#
....|...|.
..#.|...|.
..+-+-+#|.
..|.|.|.|.
.#+-^-+-+.
.+----+O#.
#+----+...
......#...
```

#### Option 4: Put an alchemical retroencabulator near the bottom left corner
```
....#.....
....+---+#
....|...|.
..#.|...|.
..+-+-+#|.
..|.|.|.|.
.#+-^-+-+.
..|...|.#.
#O+---+...
......#...
```

#### Option 5: Put the alchemical retroencabulator a bit to the right instead
```
....#.....
....+---+#
....|...|.
..#.|...|.
..+-+-+#|.
..|.|.|.|.
.#+-^-+-+.
....|.|.#.
#..O+-+...
......#...
```

#### Option 6: Put a tank of sovereign glue right next to the tank of universal solvent
```
....#.....
....+---+#
....|...|.
..#.|...|.
..+-+-+#|.
..|.|.|.|.
.#+-^-+-+.
.+----++#.
#+----++..
......#O..
```

It doesn't really matter what you choose to use as an obstacle so long as you and The Historians can put it into position without the guard noticing. The important thing is having enough options that you can find one that minimizes time paradoxes, and in this example, there are 6 different positions you could choose.

**You need to get the guard stuck in a loop by adding a single new obstruction. How many different positions could you choose for this obstruction?**

In [56]:
game_map = Map.from_file("./example.txt")
guard = Guard.from_map(game_map)
guard.look_for_loops = True

# Perform the patrol step-by-step
for state in guard.patrol():
    continue
print(f"{len(set(guard.history))=}")
print(f"{len(set(guard.evil_obstacle_positions))=}")

len(set(guard.history))=41
len(set(guard.evil_obstacle_positions))=6


In [57]:
game_map = Map.from_file("./input.txt")
guard = Guard.from_map(game_map)
guard.look_for_loops = True

# Perform the patrol step-by-step
for state in guard.patrol():
    continue
print(f"{len(set(guard.history))=}")
print(f"{len(set(guard.evil_obstacle_positions))=}")

len(set(guard.history))=4559
len(set(guard.evil_obstacle_positions))=1604
